In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
#save the urls of each of the games data tables
urlSid = "https://steamcharts.com/app/289070"
urlHum = "https://steamcharts.com/app/1124300"
urlEur = "https://steamcharts.com/app/236850"

#access the urls
SidResponse = requests.get(urlSid)
HumResponse = requests.get(urlHum)
EurResponse = requests.get(urlEur)

#Extract the html data, and use beautifulsoup to parse it
Sid_html = SidResponse.text
Sid_soup = BeautifulSoup(Sid_html, 'html.parser')

Hum_html = HumResponse.text
Hum_soup = BeautifulSoup(Hum_html, 'html.parser')

Eur_html = EurResponse.text
Eur_soup = BeautifulSoup(Eur_html, 'html.parser')


In [ ]:
#pull out the table for the civilization 6 data
Sid_table = Sid_soup.find('table')
table_data = []
if Sid_table:
    for row in Sid_table.find_all('tr'):
        row_data = []
        for cell in row.find_all(['th', 'td']):
            row_data.append(cell.get_text(strip=True))
        table_data.append(row_data)

#convert this table into a dataframe
Sid_df = pd.DataFrame(table_data[1:], columns=table_data[0])
#drop the first, unneccessary line
Sid_df = Sid_df.drop(0,axis = 0)
#create a column to identify which game this data belongs to
Sid_df['Game'] = "Civilization"
Sid_df.head()

,Month,Avg. Players,Gain,% Gain,Peak Players,Game
1,October 2025,29916.68,222.28,+0.75%,54025,Civilization
2,September 2025,29694.40,-2262.31,-7.08%,51377,Civilization
3,August 2025,31956.71,1289.87,+4.21%,52803,Civilization
4,July 2025,30666.84,2132.26,+7.47%,49480,Civilization
5,June 2025,28534.58,661.24,+2.37%,49784,Civilization


In [ ]:
#pull out the table for the Humankind data
Hum_table = Hum_soup.find('table')
table_data = []
if Hum_table:
    for row in Hum_table.find_all('tr'):
        row_data = []
        for cell in row.find_all(['th', 'td']):
            row_data.append(cell.get_text(strip=True))
        table_data.append(row_data)

#convert this table into a dataframe
Hum_df = pd.DataFrame(table_data[1:], columns=table_data[0])
#drop the first, unneccessary line
Hum_df = Hum_df.drop(0,axis = 0)
#create a column to identify which game this data belongs to
Hum_df['Game'] = "Humankind"
Hum_df.head()

,Month,Avg. Players,Gain,% Gain,Peak Players,Game
1,October 2025,476.93,9.98,+2.14%,901,Humankind
2,September 2025,466.95,-18.68,-3.85%,867,Humankind
3,August 2025,485.64,-49.40,-9.23%,885,Humankind
4,July 2025,535.04,55.20,+11.50%,1042,Humankind
5,June 2025,479.84,-10.52,-2.15%,891,Humankind


In [ ]:
#pull out the table for the Europa Universalis data
Eur_table = Eur_soup.find('table')
table_data = []
if Eur_table:
    for row in Eur_table.find_all('tr'):
        row_data = []
        for cell in row.find_all(['th', 'td']):
            row_data.append(cell.get_text(strip=True))
        table_data.append(row_data)

#convert this table into a dataframe
Eur_df = pd.DataFrame(table_data[1:], columns=table_data[0])
#drop the first, unneccessary line
Eur_df = Eur_df.drop(0,axis = 0)
#create a column to identify which game this data belongs to
Eur_df['Game'] = "Europa"
Eur_df.head()

,Month,Avg. Players,Gain,% Gain,Peak Players,Game
1,October 2025,11916.86,-497.82,-4.01%,20517,Europa
2,September 2025,12414.67,-1634.60,-11.63%,21199,Europa
3,August 2025,14049.28,656.42,+4.90%,22927,Europa
4,July 2025,13392.86,-568.20,-4.07%,21924,Europa
5,June 2025,13961.06,-886.21,-5.97%,23449,Europa


In [ ]:
#stack the three dataframes to create comprehensive dataframe
Full_df = pd.concat([Sid_df,Eur_df,Hum_df])

#define function to read the year from the "Month" column
def pull_year(row):
    return int(row['Month'][-4:])

#define function to remove the year from the "Month" column
def pull_month(row):
    return row['Month'][:-5]

#create "Year" column from the "Month" column
Full_df['Year'] = Full_df.apply(pull_year,axis = 1)

#remove the year from the "Month" column
Full_df['Month'] = Full_df.apply(pull_month,axis = 1)

#look at first couple lines to make sure everything worked
Full_df.head()

#export dataframe as csv file
Full_df.to_csv('GameData.csv', index = False)


In [ ]:
#report a summary of the dataframe dimensions
Full_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 311 entries, 1 to 51
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Month         311 non-null    object
 1   Avg. Players  311 non-null    object
 2   Gain          311 non-null    object
 3   % Gain        311 non-null    object
 4   Peak Players  311 non-null    object
 5   Game          311 non-null    object
 6   Year          311 non-null    int64 
dtypes: int64(1), object(6)
memory usage: 19.4+ KB
